In [1]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|
IDX002:DIM0016:003|004|
IDX003:DIM0256:005|006|007|008|
IDX004:DIM0016:009|010|
IDX005:DIM0016:011|012|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                   88.885935 seconds   1.31e+01 Gb GC: 4.2e+00 sec
 Add results together:                               0.257071 seconds   5.80e-02 Gb GC: 0.0e+00 sec
 In open_matvec_thread
 Setup threaded jobs:                                0.000149 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 1
 Compute matrix-vector:                              4.630280 seconds (15.19 M allocations: 729.804 MiB, 3.86% gc time, 99.71% compilation time)
 Now collect thread results :                        0.019638 seconds (1.96 k allocations: 113.500 KiB, 99.29% compilation time)
size(s_bst) = (16300, 1)
size(s_tps) = (49378, 

In [2]:
e_tpsci, v_tpsci = FermiCG.tpsci_ci(v_tps, cluster_ops, clustered_ham, thresh_cipsi=1e-4, thresh_foi=1e-6, thresh_asci=-1)
e_pt = FermiCG.compute_pt2_energy(v_tpsci, cluster_ops, clustered_ham, thresh_foi=1e-8, verbose=0);

 ci_vector     : (1, 1)
 thresh_cipsi  : 0.0001
 thresh_foi    : 1.0e-6
 thresh_asci   : -1
 thresh_var    : nothing
 max_iter      : 10
 conv_thresh   : 0.0001
 nbody         : 4
 incremental   : true
 ci_conv       : 1.0e-5
 ci_max_iter   : 50
 ci_max_ss_vecs: 12
 davidson      : false
 max_mem_ci    : 20.0
 threaded      : true


     Selected CI Iteration:    1 epsilon:   0.00010000
 Memory needed to hold full CI matrix:   0.00000001 (Gb) Max allowed:  20.00000000 (Gb)

 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =     1: 
 Build full Hamiltonian matrix with dimension:       0.399319 seconds (708.39 k allocations: 37.035 MiB, 99.67% compilation time)
 Now diagonalize
 Diagonalization time:                               0.000002 seconds
(1, 1)(1, 1)
 Compute S2 expectation values:                      0.591772 seconds (442.18 k allocations: 23.986 MiB, 99.21% compilation time)
  Root       Energy           S2
     1 -18.31710895

# BST PT2

In [24]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy_old(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-6, max_iter=100, tol=1e-9, verbose=0);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001800 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001409 seconds
 Compute <S^2>:                                      0.000330 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 100
 nbody       : 4
 thresh_foi  : 1.0e-6
 max_number  : nothing
 tol         : 1.0e-9
 opt_ref     : true
 verbose     : 0

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ================================

In [19]:
Revise.retry()

In [35]:
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-12, verbose=1);

 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 1.0e-12
 max_number  : nothing
 tol         : 1.0e-5
 opt_ref     : true
 verbose     : 1

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.002806 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001420 seconds
 Compute <S^2>:                                      0.000637 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Diagonalization time:                      

In [34]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
@time v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.003814 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001514 seconds
 Compute <S^2>:                                      0.000618 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.151910 seconds   6.24e-02 Gb GC: 0.0e+00 sec
 Add results together:                               0.053566 seconds   1.76e-02 Gb GC: 2.6e-02 sec
 Length of input      FOIS:                             16382
 Compute <X|

In [37]:
@time e_pt, v_pt = FermiCG.compute_pt1_wavefunction(s_bst, v, cluster_ops, clustered_ham);


 |...................................BST-PT2............................................
 Length of input      FOIS:                             16382
 Compute <X|H|0>:                                    0.036077 seconds   1.07e-02 Gb
 Compute <X|F|0>:                                    0.000917 seconds   7.39e-04 Gb
 State   1: E(PT2) corr:                          -0.00992614
  Root         E(0)         E(2)
     1 -18.31710895 -18.32703509
 ......................................................................................|
  0.106297 seconds (315.60 k allocations: 41.468 MiB)


# Pseudo-Canonicalization

In [38]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

v = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-3)

e_var, v = FermiCG.ci_solve(v, cluster_ops, clustered_ham); 

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)

@time v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.002983 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001062 seconds
 Compute <S^2>:                                      0.000405 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.637814 seconds   7.06e-02 Gb GC: 0.0e+00 sec
 Add results together:                               0.087764 seconds   1.10e-02 Gb GC: 3.2e-02 sec
 |== BST CI ========================================================
 Solv

In [39]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(s_bst, v, cluster_ops, clustered_ham, verbose=1);


 |...................................BST-PT2............................................
 Length of input      FOIS:                            109661
 Compute <X|H|0>:                                    8.314834 seconds   2.63e+00 Gb
 Compute <X|F|0>:                                    0.037713 seconds   1.51e-02 Gb
 State   1: E(PT2) corr:                          -0.00040339
  Root         E(0)         E(2)
     1 -18.32971586 -18.33011925
 ......................................................................................|
 10.188144 seconds (40.40 M allocations: 3.051 GiB, 5.41% gc time, 0.04% compilation time)


In [40]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(v, cluster_ops, clustered_ham, verbose=1, thresh_foi=1e-6);

 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                   16.302599 seconds   6.89e+00 Gb GC: 1.3e+00 sec
 Add results together:                               0.874782 seconds   6.38e-01 Gb GC: 8.5e-02 sec
 Compute Compressed FOIS:                           17.178031 seconds   7.53e+00 Gb

 |...................................BST-PT2............................................
 Length of input      FOIS:                            109661
 Compute <X|H|0>:                                    7.601135 seconds   2.62e+00 Gb
 Compute <X|F|0>:                                    0.035748 seconds   1.51e-02 Gb
 State   1: E(PT2) corr:                          -0.00040339
  Root         E(0)         E(2)
     1 -18.32971586 -18.33011925
 ......................................................................................|
 26.534645 seconds (107.19 M allocations: 10.035 GiB, 7.22% gc ti

In [ ]:
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-6, verbose=1);